# 生成式對話機器人

## Step1 導入相關工具包

In [1]:
!pip install datasets
!pip install transformers[torch]

In [2]:
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## Step2 下載數據集

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# ds = Dataset.load_from_disk("./alpaca_data_zh")
# ds

ds = load_dataset("shibing624/alpaca-zh", split = 'train')
ds

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 48818
})

In [5]:
import pandas as pd

df = pd.DataFrame(ds)
df.head()

,instruction,input,output
0,保持健康的三个提示。,,以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游...
1,三原色是什么？,,三原色通常指的是红色、绿色和蓝色（RGB）。它们是通过加色混合原理创建色彩的三种基础颜色。在...
2,描述原子的结构。,,原子是物质的基本单位，它由三种基本粒子组成：质子、中子和电子。质子和中子形成原子核，位于原子...
3,如何减少空气污染？,,有很多方法来减少空气污染:\n\n1. 减少机动车排放: 通过支持可再生能源来驱动的交通工具...
4,描述一次你不得不做出困难决定的经历。,,作为一名人工智能助手，我是由程序控制的，没有感情和个人意识，我的一切决策都是基于我的算法和程...


In [6]:
output_max_length = df['output'].apply(lambda x: len(x)).max()

instruction_max_length = df['instruction'].apply(lambda x: len(x)).max()


print(output_max_length, instruction_max_length)

2466 234


In [7]:
ds[0]


{'instruction': '保持健康的三个提示。',
 'input': '',
 'output': '以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。'}

## Step3 資料預處理

In [8]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-389m-zh")
tokenizer

BloomTokenizerFast(name_or_path='Langboat/bloom-389m-zh', vocab_size=42437, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)

In [9]:
"\n".join(["Human: " + ds[0]["instruction"], ds[0]["input"]]).strip() + "\n\nAssistant: "

ds[0]["output"] + tokenizer.eos_token


'以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。</s>'

In [10]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: ")
    response = tokenizer(example["output"] + tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"]

    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }



In [11]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 48818
})

In [12]:
tokenizer.decode(tokenized_ds[10]["input_ids"])

'Human: 法国的首都是什么？\n\nAssistant: 法国的首都是巴黎。</s>'

In [13]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[10]["labels"])))

'法国的首都是巴黎。</s>'

## Step4 模型創建

In [14]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-389m-zh")

## Step5 撰寫訓練計畫

In [15]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1
)


## Step6 建立訓練器

In [16]:
df = pd.DataFrame(tokenized_ds)
df["input_ids_len"] = df['input_ids'].apply(lambda x: len(x))
df.head()


,input_ids,attention_mask,labels,input_ids_len
0,"[23069, 29, 210, 6583, 24772, 8995, 13533, 671...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10...",138
1,"[23069, 29, 10138, 1691, 2029, 24252, 189, 189...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10...",177
2,"[23069, 29, 210, 9162, 1691, 7630, 6656, 671, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10...",256
3,"[23069, 29, 25182, 6189, 12793, 10848, 1476, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10...",220
4,"[23069, 29, 210, 9162, 5189, 854, 13836, 10407...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, -100, -100, -100, -100, -100, -10...",146


In [17]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)



## Step7 模型訓練

In [18]:
trainer.train()

  0%|          | 0/762 [00:00<?, ?it/s]

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 

## Step8 模型推理

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)



In [ ]:
ipt = "Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: "
pipe(ipt, max_length=256, do_sample=True)


[{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 考试技巧是一个术语，通常分为考点和策略两个部分。一个考点是指一道题或类似题，要求我们掌握相应的技巧，以确保对题目的正确性或者答案的成功性。考点通常是设定的，因此我们不需要完全理解每个考点，也能通过多种途径进行熟悉、强化并熟悉。此外，每个考点都是固定的，因此我们可以根据自己的情况变化，提高出题策略和出错率。策略是学会答题顺序，保持正确的思维格式，不歪歪、不漏漏、不放弃，避免错误的操作方式等。总之，考试技巧是掌握在考试中获胜的关键所在。'}]